# Methods and papers:
* ProtoNet (https://arxiv.org/abs/1904.04232)
* MCT + DFMN (https://arxiv.org/abs/2002.12017)

## Notes:
* ProtoNet - результаты тестирования совпадают с результатами, указанными в статье.
* MCT + DFMN - оперативной памяти не хватает для обучения в описанном сценарии на ResNet-12, ConvNet-256 или ConvNet-128; поэтому использовалась маленькая сеть ConvNet-64. Результаты тестирования совпадают с результатами для ConvNet-64, указанными в статье. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
INDEX_FILE = r"D:\petrtsv\projects\ds\torch-nn-project\index.csv"

records = pd.read_csv(INDEX_FILE, index_col=0)
records = records[
    (records['record'].isin({
        100, 110, 130, 140, 30, 710, 720, 730, 310, 370, 750, 760
    }))
]

records.head()

,Name,Explanation,backbone,batch_size,dataset,epochs,frozen_feature_extractor,lr,start_time,test_accuracy,...,session_dir,test_ts_steps,train_ts_steps,all_global_prototypes,image_size,balanced_batches,val_batch_size,dataset_classes,alpha,pretrained_model
358,Few-Shot Learning Training,NaN,NaN,16.0,miniImageNet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,Few-Shot Learning Training,NaN,NaN,16.0,miniImageNet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,FSL_MCTDFMN_055670-43-55-20-05-04-2020,NaN,NaN,8.0,miniImageNet,NaN,NaN,NaN,NaN,NaN,...,D:\petrtsv\projects\ds\pytorch-sessions\FSL_MC...,10.0,1.0,True,84.0,True,15.0,NaN,NaN,NaN
379,FSL_MCTDFMN_637276-53-47-18-06-04-2020,NaN,NaN,8.0,miniImageNet,NaN,NaN,NaN,NaN,NaN,...,D:\petrtsv\projects\ds\pytorch-sessions\FSL_MC...,10.0,1.0,True,84.0,True,15.0,NaN,NaN,NaN
380,FSL_MCTDFMN_transfer_787893-38-04-18-09-04-2020,NaN,NaN,8.0,miniImageNet->cub,NaN,NaN,NaN,NaN,NaN,...,D:\petrtsv\projects\ds\pytorch-sessions\FSL_MC...,10.0,1.0,True,84.0,True,15.0,NaN,NaN,NaN


# Accuracy scores:

In [3]:
COLUMNS = ['record', 'dataset', 'accuracy', 'feature_extractor',
           'n_shot', 'model', 'n_way', 'test_ts_steps']

scores = records[COLUMNS]
scores

,record,dataset,accuracy,feature_extractor,n_shot,model,n_way,test_ts_steps
358,30.0,miniImageNet,0.734688,resnet18,5.0,ProtoNet,5.0,NaN
359,30.0,miniImageNet,0.524479,resnet18,1.0,ProtoNet,5.0,NaN
378,100.0,miniImageNet,0.758711,conv64-np-o,5.0,MCT_DFMN,5.0,10.0
379,100.0,miniImageNet,0.665600,conv64-np-o,1.0,MCT_DFMN,5.0,10.0
380,110.0,miniImageNet->cub,0.467089,conv64-np-o,1.0,MCT_DFMN,5.0,10.0
381,110.0,miniImageNet->cub,0.619311,conv64-np-o,5.0,MCT_DFMN,5.0,10.0
383,130.0,miniImageNet,0.521622,conv64-np-o,1.0,ProtoNet,5.0,NaN
384,130.0,miniImageNet,0.687956,conv64-np-o,5.0,ProtoNet,5.0,NaN
385,140.0,miniImageNet->cub,0.415311,conv64-np-o,1.0,ProtoNet,5.0,NaN
386,140.0,miniImageNet->cub,0.640022,conv64-np-o,5.0,ProtoNet,5.0,NaN
